## Runnable Interface 介绍与使用

为了尽可能简化创建自定义链的过程，Langchain 实现了一个 **[Runnable](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)** 协议。

许多 LangChain 组件都实现了 Runnable 协议，包括 chat models, LLMs, output parsers, retrievers, prompt templates等等。此外，还有几个用于处理可运行对象的[有用原语](https://python.langchain.com/v0.1/docs/expression_language/primitives/)。

Runnable 是一个标准接口，包括：

- stream：流式返回生成内容（chunk）
- invoke：对输入调用该链
- batch：对输入列表调用该链

不同组件的输入和输出类型有所差异:

| 组件    | 输入类型                                           | 输出类型           |
| ------------ | ----------------------------------------------------- | --------------------- |
| Prompt       | Dictionary                                            | PromptValue           |
| ChatModel    | Single string, list of chat messages or a PromptValue | ChatMessage           |
| LLM          | Single string, list of chat messages or a PromptValue | String                |
| OutputParser | The output of an LLM or ChatModel                     | Depends on the parser |
| Retriever    | Single string                                         | List of Documents     |
| Tool         | Single string or dictionary, depending on the tool    | Depends on the tool   |

- **input_schema**: 用于验证输入数据的格式。
  例如对于上面的 prompt，它需要一个包含 "topic" 的字典作为输入:
  - `{"topic": "编程"}` ✓   
  - `{"subject": "编程"}` ✗  *错误:缺少必需的 topic 参数*

- **output_schema**: 用于验证输出数据的格式。
  例如对于 ChatModel:
  - 输出必须是 ChatMessage 格式 ✓
  - 输出是普通字符串 ✗  *错误:格式不符合要求*

这些 schema 通过 Pydantic 模型自动生成，可以在运行前就发现数据格式问题。

当使用 `|` 运算符组合不同的 Runnable 组件时，需要确保前一个组件的 output_schema 与后一个组件的 input_schema 相匹配。例如:

- `PromptTemplate | ChatModel`: ✓ 
  因为 PromptTemplate 输出 PromptValue，而 ChatModel 可以接受 PromptValue 作为输入

- `ChatModel | PromptTemplate`: ✗
  因为 ChatModel 输出 ChatMessage，但 PromptTemplate 需要字典作为输入

所以在使用 `|` 组合时要注意检查前后组件的 schema 是否兼容



### Input Schema

为了演示如何使用，下面我们创建一个超级简单的PromptTemplate + ChatModel链。

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import os

# 初始化模型
# 从环境变量获取 Key
api_key = os.environ.get("OPENAI_API_KEY", None)  # 如果找不到，会返回 None

# 初始化 ChatOpenAI 模型，指定使用的模型为 'gpt-4o-mini'
model = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_base="https://api.gptsapi.net/v1",
    openai_api_key=api_key,
    
    temperature=0.5,
    )
# 创建提示模板
prompt = ChatPromptTemplate.from_template("讲个关于 {topic} 的笑话吧")

# 创建一个LLMChain，将Prompt和Model绑定在一起
chain = LLMChain(prompt=prompt, llm=model)

# 执行链并传入变量
response = chain.run(topic="编程")

# 打印输出结果
print(response)


/var/folders/4h/6gp0mhzx1cd053b8d1_dm8080000gn/T/ipykernel_53902/36625536.py:22: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=model)
/var/folders/4h/6gp0mhzx1cd053b8d1_dm8080000gn/T/ipykernel_53902/36625536.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(topic="编程")


当然可以！这是一个关于编程的笑话：

为什么程序员总是喜欢在海边工作？

因为他们喜欢在“海”里“调试”！ 

（“调试”在英文中是“debug”，而“海”是“sea”，发音相似。）希望你喜欢这个笑话！


#### schema 方法

一个描述 Runnable 接受的输入的说明。这是根据任何 Runnable 结构动态生成的 Pydantic 模型。您可以调用 .schema() 来获取 `JSONSchema` 表示。

In [2]:
# 查看 Chain 的输入类型
chain.input_schema.schema()

/var/folders/4h/6gp0mhzx1cd053b8d1_dm8080000gn/T/ipykernel_53902/2593355760.py:2: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'topic': {'default': None, 'title': 'Topic'}},
 'title': 'ChainInput',
 'type': 'object'}

In [3]:
# 查看 Prompt 的输入类型（Chain的输入从 Prompt 开始，因此输入类型一致）
prompt.input_schema.schema()

/var/folders/4h/6gp0mhzx1cd053b8d1_dm8080000gn/T/ipykernel_53902/3578723451.py:2: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  prompt.input_schema.schema()


{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [4]:
# 查看 Model 的输入类型
model.input_schema.schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'response_metadata': {'title': 'Response Metadata', 'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Name'},
    'id': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     '

### Output Schema

输出类型仍然可以调用 .schema() 来获取其 `JSONSchema` 表示。

In [5]:
# 查看 Chain 的输出类型
chain.output_schema.schema()

/var/folders/4h/6gp0mhzx1cd053b8d1_dm8080000gn/T/ipykernel_53902/4013333033.py:2: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.output_schema.schema()


{'properties': {'text': {'default': None, 'title': 'Text'}},
 'title': 'ChainOutput',
 'type': 'object'}

### Stream

使用 .stream() 方法查看（同步）流式输出结果

In [6]:
for s in chain.stream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

AttributeError: 'dict' object has no attribute 'content'

### Invoke
使用 .invoke() 方法单次（同步）调用

In [7]:
chain.invoke({"topic": "程序员"})

{'topic': '程序员',
 'text': '当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进酒吧，点了一杯酒。酒保问他：“你为什么看起来这么沮丧？”\n\n程序员叹了口气，说：“我在家里修了一个bug，结果现在家里的灯全都不亮了！”\n\n酒保好奇地问：“那你怎么解决的？”\n\n程序员无奈地回答：“我把灯的开关重启了一下，结果它又亮了！”\n\n酒保笑着说：“看来你真是个‘开关’程序员啊！”\n\n希望这个笑话能让你开心！'}

### Batch
使用 .batch() 方法（同步）批量调用

In [8]:
chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[{'topic': '程序员',
  'text': '当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进酒吧，点了一杯酒。酒保问他：“你为什么看起来这么沮丧？”\n\n程序员叹了口气说：“我刚刚修复了一个bug，但又出现了两个新的bug。”\n\n酒保笑着说：“那你应该感到高兴啊！至少你在做‘加法’！”\n\n程序员摇摇头：“不，我更希望它们能‘减法’！”\n\n希望这个笑话能让你笑一笑！'},
 {'topic': '产品经理',
  'text': '当然可以！这是一个关于产品经理的笑话：\n\n为什么产品经理总是带着铅笔和纸？\n\n因为他们总是在“划重点”！'},
 {'topic': '测试经理',
  'text': '当然可以！这是一个关于测试经理的笑话：\n\n有一天，测试经理跟开发经理讨论项目进展。测试经理说：“我们现在的测试覆盖率达到了90%！”\n\n开发经理一听，立刻紧张起来，问：“那剩下的10%怎么办？”\n\n测试经理微笑着回答：“剩下的10%？那是我们留给用户的惊喜！”\n\n希望这个笑话能让你开心！'}]

In [9]:
messages = chain.batch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

In [10]:
# 使用 StrOutputParser 来处理 Batch 批量输出
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

for idx, m in enumerate(messages):
    print(f"笑话{idx}:\n")
    print(output_parser.invoke(m))
    print("\n")

笑话0:



ValidationError: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value={'topic': '程序员', 't...话能让你开心！'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

## 异步操作

这些方法也有相应的异步方法，应与 `asyncio` 的 `await` 语法一起使用以进行并发操作：

- astream：异步地流式返回生成内容（chunk）
- ainvoke：异步地对输入调用该链
- abatch：异步地对输入列表调用该链
- astream_log: 在发生时会返回中间步骤，并且最终返回结果之外。
- astream_events: beta 流式传输事件，在 langchain-core 0.1.14 中引入


### Async Stream

In [11]:
async for s in chain.astream({"topic": "程序员"}):
    print(s.content, end="", flush=True)

AttributeError: 'dict' object has no attribute 'content'

### Async Invoke

In [12]:
await chain.ainvoke({"topic": "程序员"})

{'topic': '程序员',
 'text': '当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进酒吧，点了一杯啤酒。酒保问他：“你要不要加点盐？” \n\n程序员回答：“不，我只想要一个简单的饮料，不需要任何额外的功能！”\n\n希望你喜欢这个笑话！'}

### Async Batch

In [13]:
await chain.abatch([{"topic": "程序员"}, {"topic": "产品经理"}, {"topic": "测试经理"}])

[{'topic': '程序员',
  'text': '当然可以！这是一个关于程序员的笑话：\n\n有一天，一个程序员走进一家酒吧，点了一杯酒。酒保问他：“你喝什么？”\n\n程序员回答：“我喝代码！”\n\n酒保困惑地问：“代码是什么味道的？”\n\n程序员微笑着说：“就像错误一样，喝了之后会让我头疼，但我还是忍不住想再来一杯！”\n\n希望你喜欢这个笑话！'},
 {'topic': '产品经理',
  'text': '当然可以！这是一个关于产品经理的笑话：\n\n有一天，产品经理、开发者和设计师一起去野营。晚上，他们围着篝火聊天。\n\n产品经理说：“我觉得我们应该做一个新的帐篷，功能更全，使用更方便。”\n\n开发者说：“那我们需要明确需求，才能开始开发。”\n\n设计师插嘴：“我们还得考虑用户体验，帐篷的颜色和形状也很重要！”\n\n这时，篝火熄灭了，大家都在黑暗中摸索。产品经理说：“没关系，我们可以迭代，等明天再改进。”\n\n开发者说：“可是我们现在没有光源啊！”\n\n设计师叹了口气：“看来我们需要一个 MVP（最小可行产品）——先找到火柴！”\n\n哈哈，希望这个笑话能让你开心！'},
 {'topic': '测试经理',
  'text': '当然可以！这是一个关于测试经理的笑话：\n\n有一天，测试经理在会议上说：“我们需要提高我们的测试覆盖率！”\n\n开发人员问：“那我们应该怎么做呢？”\n\n测试经理微笑着回答：“简单！我们只需要把所有的代码都写成注释，这样覆盖率就百分之百了！”\n\n大家都笑了，开发人员无奈地说：“那我们的产品还怎么用？”\n\n测试经理耸耸肩：“那就再写个测试用例，确保注释是正确的！”\n\n希望这个笑话能让你开心！'}]